## RIOT API

Import necessary libraries

In [ ]:
from fastapi import FastAPI, Request, HTTPException
from fastapi.responses import FileResponse, Response
from fastapi.middleware.cors import CORSMiddleware
from pydantic import BaseModel
from typing import List, Optional
from openai import OpenAI
from dotenv import load_dotenv
import httpx
import asyncio
import logging
import os
import requests
import pandas as pd
import cassiopeia as cass

load_dotenv()
RIOT_API_KEY = os.getenv("RIOT_API_KEY")
if not RIOT_API_KEY:
    raise ValueError("RIOT_API_KEY environment variable is not set")
RIOT_API_BASE_URL = "api.riotgames.com"

app = FastAPI()

### Dev Variables

In [ ]:
summoner_name = "gameb0x"
region = "na1"
mass_region = "americas"
no_games = 25
queue_id = 420

### Prod Variables

Retrieve API Key from https://developer.riotgames.com/\

Retrieve Queue ID: https://static.developer.riotgames.com/docs/lol/queues.json\

Queue IDs:\
400: 5v5 Draft Pick\
420: 5v5 Ranked Solo\
430: 5v5 Blind Pick\
440: 5v5 Ranked Flex\
450: 5v5 ARAM

In [ ]:
summoner_name = input("Please enter the summoner name: ")
print("Summoner Name:", summoner_name)
regions = ["na1", "eun1", "euw1", "jp1", "kr", "br1"]
region = input(
    "Please enter the region (na1, eun1, euw1, jp1, kr, br1). Default is 'na1': "
)
if region == "":
    region = "na1"
elif region not in regions:
    while region not in regions:
        region = input(
            "Invalid region. Please enter the region (na1, eun1, euw1, jp1, kr, br1): "
        )
print("Region:", region)
mass_regions = ["AMERICAS", "ASIA", "EUROPE", "SEA"]
mass_region = input(
    "Please enter the mass region (AMERICAS, ASIA, EUROPE, SEA). Default is 'AMERICAS': "
)
if mass_region == "":
    mass_region = "AMERICAS"
elif mass_region not in mass_regions:
    while mass_region not in mass_regions:
        mass_region = input(
            "Invalid mass region. Please provide a valid MASS REGION (AMERICAS, ASIA, EUROPE, SEA): "
        )
print("Mass Region:", mass_region)

## Set Routes

Riot API Route Dictionary

In [ ]:
RIOT_API_ROUTES = {
    "summoner": "/lol/summoner/v4/summoners/by-name/{summonerName}",
    "match_by_puuid": "/lol/match/v5/matches/by-puuid/{puuid}/ids",
    "match_by_id": "/lol/match/v5/matches/{matchId}",
    "match_timeline": "/lol/match/v5/matches/{matchId}/timeline"
}

Queue ID Route Dictionary

In [ ]:
QUEUE_ID_ROUTES = {
    "draft_pick": 400,
    "ranked_solo": 420,
    "blind_pick": 430,
    "ranked_flex": 440,
    "aram": 450
}

Set: RIOT_API_URL
- Using the previously supplied variables, sets the Riot API URL

In [ ]:
route = RIOT_API_ROUTES["summoner"].format(summonerName=summoner_name)
RIOT_API_URL = f"https://{region}.{RIOT_API_BASE_URL}{route}"
RIOT_API_URL

Retrieve: PLAYER_INFO & Set Variables
- Player Info Fields
  - id, accountId, puuid, name, profileIconId, revisionDate, summonerLevel
- Variables Set
  - id, account_id, puuid, name, profile_icon_id, summoner_level

In [ ]:
resp = requests.get(RIOT_API_URL, headers={"X-Riot-Token": RIOT_API_KEY})
player_info = resp.json()
id = player_info["id"]
account_id = player_info["accountId"]
puuid = player_info["puuid"]
name = player_info["name"]
profile_icon_id = player_info["profileIconId"]
summoner_level = player_info["summonerLevel"]

#### MATCH BY PUUID

Update: RIOT_API_URL
- Change region to mass_region
- Match history of the player based on puuid

In [ ]:
route = RIOT_API_ROUTES["match_by_puuid"].format(puuid=puuid)
RIOT_API_URL = f"https://{mass_region}.{RIOT_API_BASE_URL}{route}"
RIOT_API_URL

Set: MATCH_IDS

In [ ]:
resp = requests.get(RIOT_API_URL, headers={"X-Riot-Token": RIOT_API_KEY})
match_ids = resp.json()

Set: RECENT_MATCH
- Most recently played match

In [ ]:
recent_match = match_ids[0]
recent_match

Update: RIOT_API_URL 
- Route = 'match_by_id'
- Match details of the most recent match

In [ ]:
route = RIOT_API_ROUTES["match_by_id"].format(matchId=recent_match)
RIOT_API_URL = f"https://{mass_region}.{RIOT_API_BASE_URL}{route}"
RIOT_API_URL

Retrieve: Match Data
- From updated RIOT_API_URL with most recent match

In [ ]:
resp = requests.get(RIOT_API_URL, headers={"X-Riot-Token": RIOT_API_KEY})
match_data = resp.json()
match_data

Retrieve: MATCH_DATA --> Headlines
- Keys = 'metadata', 'info'

In [ ]:
match_data.keys()

Retrieve: MATCH_DATA --> METADATA
- All Participants PUUID

In [ ]:
match_data['metadata']

Retrieve: MATCH_DATA
- Dictionary Keys

In [ ]:
match_data['info'].keys()

Retrieve: Match Data
- Game Duration

In [ ]:
match_data['info']['gameDuration'] / 60

In [ ]:
player_data_00 = match_data['info']['participants'][0]
player_data_00

In [ ]:
k = player_data_00['kills']
d = player_data_00['deaths']
a = player_data_00['assists']
print("Kills:", k)
print("Deaths:", d)
print("Assists:", a)
print("KDA:", (k + a) / d)

### Our Player

Code is incorrect below

In [ ]:
def get_player_index(match_data, puuid):
    # Get player index from match data
    participants = match_data['metadata']['participants']
    player_index = participants.index(puuid)
    return player_index

def get_match_data(match_id):
    match_data = get_match(match_id)
    return match_data

def get_player_index(match_data, puuid):
    participants = match_data['metadata']['participants']
    player_index = participants.index(puuid)
    return player_index

def get_player_data(match_data, player_index):
    player_data = match_data['info']['participants'][player_index]
    return player_data

def get_kda(player_data):
    k = player_data['kills']
    d = player_data['deaths']
    a = player_data['assists']
    kda = (k + a) / d
    return kda

def get_win(player_data):
    win = player_data['win']
    return win

def get_position(player_data):
    position = player_data['teamPosition']
    return position

def get_champion(player_data):
    champion = player_data['championName']
    return champion

def get_summoner_name(player_data):
    summoner_name = player_data['summonerName']
    return summoner_name

def get_game_duration(match_data):
    game_duration = match_data['info']['gameDuration'] / 60
    return game_duration

def get_player_data(match_id, puuid):
    match_data = get_match_data(match_id)
    player_index = get_player_index(match_data, puuid)
    player_data = get_player_data(match_data, player_index)
    return player_data

def get_player_stats(match_id, puuid):
    player_data = get_player_data(match_id, puuid)
    kda = get_kda(player_data)
    win = get_win(player_data)
    position = get_position(player_data)
    champion = get_champion(player_data)
    summoner_name = get_summoner_name(player_data)
    game_duration = get_game_duration(match_data)
    return kda, win, position, champion, summoner_name, game_duration

kda, win, position, champion, summoner_name, game_duration = get_player_stats(match_id, puuid)

Update: PLAYER_INDEX
- Locate our player in the match data using PUUID

In [ ]:
# List of participant puuids
participants = match_data['metadata']['participants']
# Find where in the data our players puuid is found
player_index = participants.index(puuid)
player_index
participants[player_index]

Retrieve: summonerName

In [ ]:
match_data['info']['participants'][player_index]['summonerName']

Set: PLAYER_DATA
- championName, position, kills, deaths, assists, win

In [ ]:
player_data = match_data['info']['participants'][player_index]

champion = player_data['championName']
position = player_data['teamPosition']
k = player_data['kills']
d = player_data['deaths']
a = player_data['assists']
kda = (k + a) / d
win = player_data['win']

print("Champion:", champion, "Kills:", k, "Deaths:", d, "Assists:", a, "Win:", win)
print("KDA:", kda)

### FASTAPI

GET: PUUID
- From SUMMONER_NAME

In [ ]:
@app.get("/puuid/{summoner_name}")
async def get_puuid(summoner_name: str, region: str, RIOT_API_KEY: str):
    RIOT_API_URL = f"https://{region}.api.riotgames.com/lol/summoner/v4/summoners/by-name/{summoner_name}"
    async with httpx.AsyncClient() as client:
        response = await client.get(RIOT_API_URL, headers={"X-Riot-Token": RIOT_API_KEY})
        
        if response.status_code != 200:
            raise HTTPException(status_code=response.status_code, detail=response.text)
        
        player_info = response.json()

        puuid = player_info['puuid']
    
    return puuid
puuid = await get_puuid(summoner_name, region, RIOT_API_KEY)
print(puuid)

GET MATCH_IDS
- List of all the match IDs given a players puuid and mass region

In [ ]:
@app.get("/match_ids/{puuid}")
async def get_match_ids(puuid: str, mass_region: str, RIOT_API_KEY: str):
    RIOT_API_URL = f"https://{mass_region}.api.riotgames.com/lol/match/v5/matches/by-puuid/{puuid}/ids"
    async with httpx.AsyncClient() as client:
        response = await client.get(RIOT_API_URL, headers={"X-Riot-Token": RIOT_API_KEY})
        
        if response.status_code != 200:
            raise HTTPException(status_code=response.status_code, detail=response.text)
        
        match_ids = response.json()
        # return {"match_ids": match_ids}
    return match_ids
    
match_ids = await get_match_ids(puuid, mass_region, RIOT_API_KEY)
print (match_ids)

GET: MATCH_DATA

In [ ]:
@app.get("/match_data/{match_id}")
async def get_match_data(match_id: str, mass_region: str, RIOT_API_KEY: str):
    RIOT_API_URL = f"https://{mass_region}.api.riotgames.com/lol/match/v5/matches/{match_id}"
    async with httpx.AsyncClient() as client:
        response = await client.get(RIOT_API_URL, headers={"X-Riot-Token": RIOT_API_KEY})
        
        if response.status_code != 200:
            raise HTTPException(status_code=response.status_code, detail=response.text)
        
        match_data = response.json()
        # return {"match_data": match_data}
    return match_data

match_data = await get_match_data(match_ids[0], mass_region, RIOT_API_KEY)
print(match_data)

Retrieve: Our Player's Data

In [33]:
@app.get("/player_data/{puuid}")
async def find_player_data(puuid: str, match_id: str, mass_region: str, RIOT_API_KEY: str):
    RIOT_API_URL = f"https://{mass_region}.api.riotgames.com/lol/match/v5/matches/{match_id}"
    async with httpx.AsyncClient() as client:
        response = await client.get(RIOT_API_URL, headers={"X-Riot-Token": RIOT_API_KEY})
        
        if response.status_code != 200:
            raise HTTPException(status_code=response.status_code, detail=response.text)
        
        match_data = response.json()
        participants = match_data['metadata']['participants']
        player_index = participants.index(puuid)
        player_data = match_data['info']['participants'][player_index]
        
    return player_data

player_data = await find_player_data(puuid, match_ids[0], mass_region, RIOT_API_KEY)
print(player_data)

{'allInPings': 0, 'assistMePings': 0, 'assists': 5, 'baitPings': 0, 'baronKills': 0, 'basicPings': 0, 'bountyLevel': 0, 'challenges': {'12AssistStreakCount': 0, 'abilityUses': 131, 'acesBefore15Minutes': 0, 'alliedJungleMonsterKills': 0, 'baronTakedowns': 0, 'blastConeOppositeOpponentCount': 0, 'bountyGold': 300, 'buffsStolen': 1, 'completeSupportQuestInTime': 0, 'controlWardsPlaced': 0, 'damagePerMinute': 744.8624185806431, 'damageTakenOnTeamPercentage': 0.17705760315919739, 'dancedWithRiftHerald': 0, 'deathsByEnemyChamps': 6, 'dodgeSkillShotsSmallWindow': 0, 'doubleAces': 0, 'dragonTakedowns': 2, 'earliestDragonTakedown': 1332.3352184, 'earliestElderDragon': 1720.5834390999999, 'earlyLaningPhaseGoldExpAdvantage': 0, 'effectiveHealAndShielding': 0, 'elderDragonKillsWithOpposingSoul': 0, 'elderDragonMultikills': 0, 'enemyChampionImmobilizations': 20, 'enemyJungleMonsterKills': 1, 'epicMonsterKillsNearEnemyJungler': 0, 'epicMonsterKillsWithin30SecondsOfSpawn': 0, 'epicMonsterSteals': 0,

In [34]:
def find_player_data(match_data, puuid):
    participants = match_data["metadata"]["participants"]
    player_index = participants.index(puuid)
    player_data = match_data["info"]["participants"][player_index]
    return player_data


find_player_data(match_data, puuid)

{'allInPings': 0,
 'assistMePings': 0,
 'assists': 5,
 'baitPings': 0,
 'baronKills': 0,
 'basicPings': 0,
 'bountyLevel': 0,
 'challenges': {'12AssistStreakCount': 0,
  'abilityUses': 131,
  'acesBefore15Minutes': 0,
  'alliedJungleMonsterKills': 0,
  'baronTakedowns': 0,
  'blastConeOppositeOpponentCount': 0,
  'bountyGold': 300,
  'buffsStolen': 1,
  'completeSupportQuestInTime': 0,
  'controlWardsPlaced': 0,
  'damagePerMinute': 744.8624185806431,
  'damageTakenOnTeamPercentage': 0.17705760315919739,
  'dancedWithRiftHerald': 0,
  'deathsByEnemyChamps': 6,
  'dodgeSkillShotsSmallWindow': 0,
  'doubleAces': 0,
  'dragonTakedowns': 2,
  'earliestDragonTakedown': 1332.3352184,
  'earliestElderDragon': 1720.5834390999999,
  'earlyLaningPhaseGoldExpAdvantage': 0,
  'effectiveHealAndShielding': 0,
  'elderDragonKillsWithOpposingSoul': 0,
  'elderDragonMultikills': 0,
  'enemyChampionImmobilizations': 20,
  'enemyJungleMonsterKills': 1,
  'epicMonsterKillsNearEnemyJungler': 0,
  'epicMons

Store Data for Past X Games
- Initialize Empty Dictionary
- Loop through Match IDs
- Retrieve Match Data


In [ ]:
# Create empty dictionary to store our data
data = {
    'champion': [],
    'kills': [],
    'deaths': [],
    'assists': [],
    'win': []
}

async def gather_data():
    for match_id in match_ids:
        print(match_id)
        
        # Run two functions to get the player data from the match ID
        match_data = await get_match_data(match_id, mass_region, RIOT_API_KEY)
        player_data = find_player_data(match_data, puuid)
        
        # Assist variables we want to keep track of
        champion = player_data['championName']
        k = player_data['kills']
        d = player_data['deaths']
        a = player_data['assists']
        win = player_data['win']
         
        # add them to our dataset
        data['champion'].append(champion)
        data['kills'].append(k)
        data['deaths'].append(d)
        data['assists'].append(a)
        data['win'].append(win)

# Run the async function
await gather_data()

#### Set Variable: Data
Retrieve data on the last 20 games for the account


In [ ]:
data

# Convert Data Dictionary into a Dataframe
df = pd.DataFrame(data)

Convert: Function to call later

In [ ]:
def gather_all_data(puuid, match_ids, mass_region, RIOT_API_KEY):
    # We initialise an empty dictionary to store data for each game
    data = {"champion": [], "kills": [], "deaths": [], "assists": [], "win": []}

    for match_id in match_ids:
        print(match_id)

        # run the two functions to get the player data from the match ID
        match_data = get_match_data(match_id, mass_region, RIOT_API_KEY)
        if "status" in match_data and match_data["status"]["status_code"] == 404:
            print(
                f"Match data not found for match ID: {match_id}. Skipping this match ID."
            )
            continue
        player_data = find_player_data(match_data, puuid)

        # assign the variables we're interested in
        champion = player_data["championName"]
        k = player_data["kills"]
        d = player_data["deaths"]
        a = player_data["assists"]
        win = player_data["win"]

        # add them to our dataset
        data["champion"].append(champion)
        data["kills"].append(k)
        data["deaths"].append(d)
        data["assists"].append(a)
        data["win"].append(win)

    df = pd.DataFrame(data)

    return df


# Update DF
df = gather_all_data(puuid, match_ids, mass_region, RIOT_API_KEY)

##### Dataframe Types

In [ ]:
# Converting from outcome column from 'bool' to 'int'
df["win"] = df["win"].astype(int)
# Find the averages. 'numeric_only' prevents trying to average the "champion" column
df.mean(numeric_only=True)
# Get the averages per champion
df.groupby("champion").mean()
# Order your games by amount of kills
df.sort_values("kills")

### Include Arguments
'no_games' \
'queue_id'

In [ ]:
# Updated function where you can set which queue to take data from
def get_match_ids(puuid, mass_region, no_games, queue_id, RIOT_API_KEY):
    RIOT_API_URL = (
        "https://" +
        mass_region +
        ".api.riotgames.com/lol/match/v5/matches/by-puuid/" +
        puuid + 
        "/ids?start=0" + 
        "&count=" +
        str(no_games) + 
        "&queue=" + 
        str(queue_id) + 
        "&api_key=" + 
        RIOT_API_KEY
    )
    
    print(RIOT_API_URL)
    
    resp = requests.get(RIOT_API_URL)
    match_ids = resp.json()
    return match_ids      

### Wrapping All Functions Into One

In [ ]:
def master_function(summoner_name, region, mass_region, no_games, queue_id, RIOT_API_KEY):
    puuid = get_puuid(summoner_name, region, RIOT_API_KEY)
    match_ids = get_match_ids(puuid, mass_region, no_games, queue_id, RIOT_API_KEY)
    df = gather_all_data(puuid, match_ids, mass_region, RIOT_API_KEY)
    return df

In [ ]:
def find_player_data(match_data, puuid):
    participants = match_data["metadata"]["participants"]
    player_index = participants.index(puuid)
    player_data = match_data["info"]["participants"][player_index]
    return player_data


df = master_function(
    summoner_name, region, mass_region, no_games, queue_id, RIOT_API_KEY
)

In [ ]:
df

In [ ]:
print("Summoner:", summoner_name, "of", region.upper()) # upper simply capitalises the region
print("Here are some interesting statistics about your last", no_games, "solo ranked games")

# create a count column
df['count'] = 1 

# the "agg" allows us to get the average of every column but sum the count                                       # see?
champ_df = df.groupby('champion').agg({'kills': 'mean', 'deaths': 'mean', 'assists': 'mean', 'win': 'mean', 'count': 'sum'})

# we reset in the index so we can still use the "champion" column
champ_df.reset_index(inplace=True)

# we limit it to only champions where you've played 2 or more games
champ_df = champ_df[champ_df['count'] >= 2]

# create a kda column
champ_df['kda'] = (champ_df['kills'] + champ_df['assists']) / champ_df['deaths']

# sort the table by KDA, starting from the highest
champ_df = champ_df.sort_values('kda', ascending=False) # ascending determines whether it's highest to lowest or vice-versa

# assign the first row and last row to a variable so we can print information about it
best_row = champ_df.iloc[0] # .iloc[0] simply takes the first row in dataframe
worst_row = champ_df.iloc[-1] # .iloc[-1] takes the last row in a dataframe

print("Your best KDA is on", best_row['champion'], "with a KDA of", best_row['kda'], "over", best_row['count'], "game/s")
print("Your worst KDA is on", worst_row['champion'], "with a KDA of", worst_row['kda'], "over", worst_row['count'], "game/s")

# sort by count instead
champ_df = champ_df.sort_values('count', ascending=False)

# get your most played champ
row = champ_df.iloc[0]

# Assign and format the win rate
win_rate = row['win']
win_rate = str(round(win_rate * 100, 1)) + "%"

print("Your highest played Champion is", row['champion'], "with", row['count'], 'game/s', 
     "and an average Win Rate of", win_rate)

# Sort by highest kills in a game (note, not using the champ_df groupby anymore but the raw data)
highest_kills = df.sort_values('kills', ascending=False)
row = highest_kills.iloc[0]
print("Your highest kill game was with", row['champion'], "where you had", row['kills'], "kills")


## Next Steps
- Add a new arguments to the functions. For example, you could add a "queue" argument to the get_match_history function, so you can choose which queue you want to get data from.
- Create a new function to get a players Champion Mastery, add a line to our final output saying who the highest mastery champion is
- Jump into the "match timeline" API and get some time specific data, like how many kills you average at 7 minutes
- From the LEAGUE-V4 API, grab 100 players from a league of your choice. Build a loop that gathers data about all 100 players recent games. Find out what the average number of kills a player has at 15 minutes for each role.

- How much Mastery a player has on each Champion
- In-depth game detail for every minute of the game (i.e. how much Gold/XP each player has at 12 minutes)
- In-depth objective and kill data, like who killed who, when and where
- Ranked information, such as their current rank for each queue
- Who is currently in Challenger, Grand Master & Master (& every queue below that too!)
- And much more...!

### Advanced Ideas

- Advanced error handling, 429 Rate Limits is just one of many and each requires it's own logic
- Speeding up your code using async/multiprocessing/threading

